In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.linalg as la
import scipy.sparse as sparse
from matplotlib import cm
import timeit
import numba
from numba import jit

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
def u_analytical(x, y):

    uan = np.sin(np.pi*x)*np.sin(np.pi*y) + x

    return uan

def source(x, y):

    f = -2*np.pi**2*np.sin(np.pi*x)*np.sin(np.pi*y)

    return f

In [4]:
# @jit(nopython=True)
def sor_iteration(matrix, source, grid, init_guess=None, boundary=((0, 0), (0,0)), tolerance=1.e-8, itermax=1000, omega=1.5):

    M, f, u0, B, tol, kmax = matrix, source, init_guess, boundary, tolerance, itermax

    x, y = grid

    Nx, Ny = x.shape[1]-1, y.shape[0]-1

    dx, dy = (x[0, -1] - x[0, 0])/Nx, (y[-1, 0] - y[0, 0])/Ny

    if init_guess is None:

        u0 = np.ones_like(f)

    
    u = u0.copy()

    #assign dirichlet boundary conditions
    u[0, :] = B[1][0]
    u[-1, :] = B[1][1]
    
    # u[:, 0] = B[0][0]
    # u[:, -1] = B[0][1]

    g_I = B[0][0]
    g_F = B[0][1]

    k = 0
    rel_diff = tol + 1
    conv_hist = []
       
    #iteration loop using nested loops.
    while  k < kmax and rel_diff > tol:

        print(k)

        u_next = u.copy()

        

        

        for j in range(1, Ny):
            for i in range(0, Nx+1):
                
                if i == 0:
                
                    u_next[j, i] = (1-omega)*u[j,i] + omega*(2*dy**2*(u[j, i+1] - dx*g_I[j]) + 
                                    (u_next[j-1, i] + u[j+1, i])*dx**2 - f[j,i]*dx**2*dy**2)/(2*(dx**2+dy**2))
                
                elif i == Nx: 
                    
                    u_next[j, i] = (1-omega)*u[j,i] + omega*(2*dy**2*(u_next[j, i-1] + dx*g_F[j]) + 
                                    (u_next[j-1, i] + u[j+1, i])*dx**2 - f[j,i]*dx**2*dy**2)/(2*(dx**2+dy**2))

                else:

                    u_next[j, i] = (1-omega)*u[j,i] + omega*((u_next[j, i-1] + u[j, i+1])*dy**2 + 
                                    (u_next[j-1, i] + u[j+1, i])*dx**2 - f[j,i]*dx**2*dy**2)/(2*(dx**2+dy**2))

                    


        rel_diff = la.norm(u_next-u)/la.norm(u)
        conv_hist.append(rel_diff)
        u = u_next
        k += 1

    return u, k, rel_diff, conv_hist

In [5]:
Nx = 1000
Ny = 1000

x_I, x_F = 0, 1
y_I, y_F = 0, 1

X = np.linspace(x_I, x_F, Nx+1)
Y = np.linspace(y_I, y_F, Ny+1)

dx = (x_F-x_I)/(Nx+1)
dy = (y_F-y_I)/(Ny+1)

x, y = np.meshgrid(X, Y)

In [6]:
#Set up the solver parameters

f = source(x, y)                                          #source term

u_guess = np.exp(x)#np.sin(np.pi*x/5)*np.cos(np.pi*y)*y + 100             #initial guess

itermax = 2000                                         #maximum number of iterations

tol = 1e-8                                                #desired tolerance

omega = 2/(1+np.sin(np.pi/Nx))

g_I = np.pi*np.sin(np.pi*Y) + 1
g_F = -np.pi*np.sin(np.pi*Y) + 1
boundary = ((g_I, g_F), (X, X))                                #dirichlet boundary conditions
# print(omega_opt)

In [7]:
start = timeit.default_timer()            #time the solver

u, iternum, rel_diff, conv_hist = sor_iteration(np.eye(Nx), f, (x, y), init_guess=u_guess, 
                                                boundary=boundary, tolerance=tol, itermax=itermax, omega=omega)       

elapsed = timeit.default_timer() - start

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209


KeyboardInterrupt: 

In [8]:
#analytical solution for comparison 
u_an = u_analytical(x, y)                
error = la.norm(u - u_an, 2)/la.norm(u_an, 2)

In [9]:
if iternum == itermax:

    print('WARNING: desired tolerance has not been reached for the given maximum iterations \n')

print('Omega = {:1.4f} \n'
      'Number of iterations: {} \n'
      'Last relative difference: {:1.4e} \n'
      'Error to analytical: {:1.4E} \n'
      'Time elapsed: {:1.2e} s'.format(omega, iternum, rel_diff, error, elapsed))
print('-'*50)


Omega = 1.9937 
Number of iterations: 2000 
Last relative difference: 1.8266e-04 
Error to analytical: 3.2383E-02 
Time elapsed: 4.27e+01 s
--------------------------------------------------


In [10]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


scat = ax.scatter(x, y, u, c = 'r', label = 'numerical', alpha = 1)
surf = ax.plot_surface(x, y, u_an, cmap = cm.coolwarm, label = 'analytical', alpha = 0.3)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('$u = \sin\pi x \sin\pi y + x$')
ax.set_zlim(top=2., bottom=-0.5)

surf._facecolors2d=surf._facecolors3d
surf._edgecolors2d=surf._edgecolors3d
ax.legend(loc = 2)
plt.show()